In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import datasets

In [ ]:
boston_housing = datasets.load_boston()

In [ ]:
boston_housing.feature_names

In [ ]:
# Split data and train model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(boston_housing.data, boston_housing.target, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
from joblib import dump, load
dump(model, '../models/boston_housing.joblib') 

In [ ]:
# Convert into ONNX format with onnxmltools
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([1, 13]))]
onx = convert_sklearn(model, initial_types=initial_type)
with open("boston_housting.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy as np
sess = rt.InferenceSession("boston_housting.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]